## Import and Load data file

In [6]:
# -*- coding: utf-8 -*-
import nltk
from tensorflow.python.keras.layers import LayerNormalization
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
from tensorflow.keras.models import Sequential
import numpy as np
#from keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
import codecs


words=[] #liste des mots des patterns
classes = [] # va contenir les tags
documents = [] #va contenir liste des sentences d'un pattern suivi de son tag
ignore_words = ['?', '!','-']
#data_file = codecs.open('intents.json',encoding='utf-8').read()
#data_file = open('intents.json').read()
data_file = codecs.open('intents.json',encoding='utf-8').read()
intents = json.loads(data_file)

FileNotFoundError: [Errno 2] No such file or directory: 'intents.json'

## Pre-process data

Here we iterate through the patterns and tokenize the sentence using nltk.word_tokenize() function and append each word in the words list. We also create a list of classes for our tags.

In [7]:


from nltk.tokenize import word_tokenize
from nltk import RegexpTokenizer
toknizer = RegexpTokenizer(r'''\w'|\w+|[^\w\s]''')

#ignore_words = ['?', '!','.',',','-','_','\'']
#data_file = open('intents.json').read()
data_file = codecs.open('intents.json',encoding='utf-8').read()
intents = json.loads(data_file)
#intents is a dictionnary sous forme {"":""} intents is key and we have one array as value
for intent in intents['intents']: # parcours d'un dictionnaire: name_dic['key']          #print(intent)
    for pattern in intent['patterns']:      #print(pattern)
        
   
        #tokenize each word
        #w=toknizer.tokenize(pattern)
        w = nltk.word_tokenize(pattern,language ='french') #  w est une liste des mots
        print(w)
        words.extend(w) # ajouter chaque w à la liste words ;nous avons utilisé extend car w est une liste: fusionner deux listes
        #add documents in the corpus
        documents.append((w, intent['tag'])) #ajouter une tuple (w et sa correspondence tag)dans docueents à la fin

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

FileNotFoundError: [Errno 2] No such file or directory: 'intents.json'

lemmatize each word and remove duplicate words from the list

In [8]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

0 documents
0 classes []
0 unique lemmatized words []


## Create training and testing data

In [9]:

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    #print(output_row)
    
    training.append([bag, output_row])
    #print(training)
# shuffle our features and turn into np.array
random.shuffle(training)  #Shuffle a list (reorganize the order of the list items): aléatoire,la liste va changer ne trvaille pas sur une copié
training = np.array(training)
#print(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
#print(train_x)
train_y = list(training[:,1])
print("Training data created")

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

## Create Model

In [ ]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(codecs.open('intents.json',encoding='utf-8').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [ ]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

In [ ]:
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [ ]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [ ]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [ ]:
def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

## GUI Interface

In [ ]:
#Creating GUI with tkinter
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Chatty: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

root = Tk()
root.title("Chatty")
root.geometry("400x500")
root.resizable(width=FALSE, height=FALSE)


#Create Chat window
ChatLog = Text(root, bd=0, bg="white", height="8", width="70", font="Arial",)

ChatLog.config(state=DISABLED)
ChatLog.config(background='#d3f0f0')

#Bind scrollbar to Chat window
scrollbar = Scrollbar(root, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(root, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#c27ef4",activebackground="#fcdbf4",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(root, bd=0, bg="white",width="29", height="5", font="Arial", background="#e0cefb")
#EntryBox.bind("<Return>", send)

#Place all components on the screen
scrollbar.place(x=376,y=6, height=300)
ChatLog.place(x=8,y=9, height=386, width=370)
EntryBox.place(x=130, y=401, height=40, width=265)
SendButton.place(x=6, y=401, height=40)
root.mainloop()
